In [26]:
from sklearn.linear_model            import LogisticRegression
from sklearn.model_selection         import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics                 import accuracy_score, recall_score, roc_auc_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag                      import Okt; t=Okt()
import warnings
warnings.filterwarnings(action='ignore')

In [1]:
train = pd.read_csv('./new_train.csv', index_col=0)
dev = pd.read_csv('../../data/dev.hate.csv')
test = pd.read_csv('../../data/test.hate.no_label.csv')
train.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [2]:
train.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [3]:
train = pd.concat([train, dev], axis=0)
train.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8367 entries, 0 to 470
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  8367 non-null   object
 1   label     8367 non-null   object
dtypes: object(2)
memory usage: 196.1+ KB


In [5]:
# 트레인 데이터 레이블대로 분류
none_df = train[train['label'] == 'none']['comments']
offensive_df = train[train['label'] == 'offensive']['comments']
hate_df = train[train['label'] == 'hate']['comments']
print(len(none_df), len(offensive_df), len(hate_df), "\n",
      "sum: {}".format(sum([len(none_df), len(offensive_df), len(hate_df)])))

3644 2691 2032 
 sum: 8367


In [6]:
# TfIdf 벡터라이즈
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# 훈련 데이터 전체 벡터라이즈 fit
tfidf_vectorizer.fit(train['comments'])

# 각각 레이블 벡터라이즈 transform
none_matrix = tfidf_vectorizer.transform(none_df)
offensive_matrix = tfidf_vectorizer.transform(offensive_df)
hate_matrix = tfidf_vectorizer.transform(hate_df)

none_matrix.shape, offensive_matrix.shape, hate_matrix.shape

((3644, 38992), (2691, 38992), (2032, 38992))

In [7]:
# 각각의 평균 벡터값(위치) 산출
none_vec = []
offensive_vec = []
hate_vec = []

for i in range(none_matrix.shape[1]):
    none_vec.append(none_matrix[:,i].mean())

for i in range(offensive_matrix.shape[1]):
    offensive_vec.append(offensive_matrix[:,i].mean())
    
for i in range(hate_matrix.shape[1]):
    hate_vec.append(hate_matrix[:,i].mean())

# 벡터라이즈 잘 되었는지 길이 확인
len(none_vec), len(offensive_vec), len(hate_vec)

(38992, 38992, 38992)

In [8]:
# 어레이 형태로 변환
none_vec = np.array(none_vec)
offensive_vec = np.array(offensive_vec)
hate_vec = np.array(hate_vec)
# 2차원 어레이로 변환
none_vec = none_vec.reshape(1, -1)
offensive_vec = offensive_vec.reshape(1, -1)
hate_vec = hate_vec.reshape(1, -1)

# 테스트 코멘트 벡터라이즈 적용
test_matrix = tfidf_vectorizer.transform(test['comments'])
test_matrix.shape

(974, 38992)

In [10]:
# 코멘트 <-> 각 레이블 간의 유사도 측정 후 가장 유사한 레이블 산출
from sklearn.metrics.pairwise import cosine_similarity

preds = []

for i in range(test_matrix.shape[0]):
    distances = {cosine_similarity(test_matrix[i,:], none_vec)[0][0] : "none",
                 cosine_similarity(test_matrix[i,:], offensive_vec)[0][0] : "offensive",
                 cosine_similarity(test_matrix[i,:], hate_vec)[0][0] : "hate"}
    preds.append( distances[max(distances.keys())] )
    
preds

['none',
 'none',
 'none',
 'none',
 'offensive',
 'none',
 'offensive',
 'offensive',
 'none',
 'offensive',
 'hate',
 'none',
 'offensive',
 'hate',
 'hate',
 'offensive',
 'offensive',
 'offensive',
 'hate',
 'hate',
 'offensive',
 'none',
 'offensive',
 'none',
 'none',
 'none',
 'offensive',
 'hate',
 'offensive',
 'none',
 'hate',
 'none',
 'hate',
 'offensive',
 'offensive',
 'none',
 'hate',
 'hate',
 'offensive',
 'none',
 'offensive',
 'hate',
 'hate',
 'none',
 'none',
 'offensive',
 'none',
 'offensive',
 'none',
 'hate',
 'hate',
 'hate',
 'none',
 'offensive',
 'hate',
 'hate',
 'none',
 'hate',
 'hate',
 'hate',
 'offensive',
 'none',
 'hate',
 'hate',
 'none',
 'hate',
 'none',
 'hate',
 'offensive',
 'hate',
 'none',
 'offensive',
 'offensive',
 'offensive',
 'offensive',
 'none',
 'none',
 'hate',
 'hate',
 'none',
 'hate',
 'none',
 'none',
 'none',
 'offensive',
 'offensive',
 'none',
 'hate',
 'hate',
 'offensive',
 'offensive',
 'offensive',
 'none',
 'offensive',

In [11]:
test['label'] = preds
test.head()

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,none
1,둘다 넘 좋다~행복하세요,none
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,none
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,none
4,장현승 얘도 참 이젠 짠하다...,offensive


In [12]:
def kaggle_format(df):
    df['label'][df['label'] == 'none'] = 0
    df['label'][df['label'] == 'offensive'] = 1
    df['label'][df['label'] == 'hate'] = 2
    return df

In [13]:
kaggle_format(test)

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,1
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,2
972,입에 손가릭이 10개 있으니 징그럽다,0


In [14]:
test.head()

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,1


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974 entries, 0 to 973
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  974 non-null    object
 1   label     974 non-null    object
dtypes: object(2)
memory usage: 15.3+ KB


In [17]:
test.to_csv('./0116_cos_jc.csv', index=False)

In [18]:
pd.read_csv('./0116_cos_jc.csv')

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,1
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,2
972,입에 손가릭이 10개 있으니 징그럽다,0


dd